In [ ]:
!pip3 install tensorflow opencv-python mediapipe scikit-learn matplotlib

In [8]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [19]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [20]:
def mediapipe_detection(image,model):
    image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB) 
    image.flags.writeable = False   # Image is no longer writable
    results = model.process(image) # Detecting or predicting 
    image.flags.writeable = True    # Image is writable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image,results

In [5]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION) #Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) #Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) #Draw Left Hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) #Draw Right Hand connections

In [21]:
def draw_styled_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
                             mp_drawing.DrawingSpec(thickness=1, circle_radius=1),
                             mp_drawing.DrawingSpec(thickness=1, circle_radius=1)
                             ) #Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) #Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) #Draw Left Hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) #Draw Right Hand connections

In [ ]:
#duke perdorur opencv marrim akses ne kamere
cap = cv2.VideoCapture(0)
# Set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        #read feed
        ret, frame = cap.read() #fotoja ne webcam
    
        # Make detection 
        image, results = mediapipe_detection(frame, holistic)
        print(results)

        # Draw landmarks
        draw_styled_landmarks(image,results)
        
        #show to user/screen
        cv2.imshow("OpenCV Feed", image)
    
        #breaking gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break 
    cap.release()
    cv2.destroyAllWindows()
    # diskutabile duhet me provu ne rast se nuk mbyllet kamera mire - cv2.waitKey(1)

In [22]:
#Extracting keypoint values from the pose and hand landmarks 
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose,face,lh,rh])

In [22]:
len(results.left_hand_landmarks.landmark)*3 #kalkulimi i 0 qe na duhen per secilen pjese fytyre, poze, duar
#secila landmark ka 3 variable x y dhe z, i kemi 21 landmarks te ndryshme nese e shumezojme 
# me 3 (per x, y, z) athere na jep 63 dhe neve na duhet nje array me 63 vlera qe na duhet me kthy null nese nuk kemi landmark/hand ne frame 

SyntaxError: invalid syntax (3860930966.py, line 1)

In [23]:
#Setup folders for collection
DATA_PATH = os.path.join("MP_Data")

#Veprimet qe mundohemi me i detektu
actions = np.array(['A', 'B', 'C','Ç', 'D',])
                   #  'Dh', 'E','Ë', 'F', 'G','Gj', 'H', 'I', 'J', 'K', 'L','Ll', 'M', 
                   #  'N','Nj', 'O', 'P', 'Q', 'R','Rr', 'S','Sh', 'T','Th', 'U', 'V', 'X','Xh', 'Y', 'Z','Zh',
                   # 'Përshendetje','Mirë','Faleminderit', 'Më fal','Të dua','Nuk e di' ,'Asgjë', 'Kafe', 'Çaj','Ujë',
                   #  'Çokollatë', 'Unë','Dua', 'Akullore', 'Kompjuteri', 'Telefon i menqur','Interneti', 'Vrapoj',
                   #  'Eci'
                   # ])

no_sequences = 30
sequence_length = 30
start_folder = 30 

In [15]:
#Ruajtja e seciles frame ne secilin folder te vetin
for action in actions: 
    for sequence in range(no_sequences):
        try: 
            os.makedirs(os.path.join(DATA_PATH,action, str(sequence)))
        except:
            pass

In [ ]:
#Marrja e te dhenave 
#duke perdorur opencv marrim akses ne kamere
cap = cv2.VideoCapture(0)
# Set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    #loop through actions
    #per secilin veprim i marrim nga 30 frames/videos
    for action in actions:
        for sequence in range(no_sequences):
            for frame_num in range(sequence_length):
                
                #read feed
                ret, frame = cap.read() #fotoja ne webcam

                # Make detection 
                image, results = mediapipe_detection(frame, holistic)
                print(results)
        
                # Draw landmarks
                draw_styled_landmarks(image,results)

                #Apply wait logic
                if frame_num == 0:
                    cv2.putText(image, 'Starting collection',
                                (120,200),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),1,cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video number {}'.format(action,sequence),
                                (15,12),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,255),1,cv2.LINE_AA)
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(2000)
                else:
                    cv2.putText(image, 'Collecting frames for {} Video number {}'.format(action,sequence),
                                (15,12),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,255),1,cv2.LINE_AA)

                #extract keypoints 
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH,action,str(sequence),str(frame_num))
                np.save(npy_path, keypoints)
                
               
                #breaking gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break 
    cap.release()
    cv2.destroyAllWindows()
    # diskutabile duhet me provu ne rast se nuk mbyllet kamera mire - cv2.waitKey(1)

In [24]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [25]:
label_map = {label:num for num, label in enumerate(actions)}

In [25]:
sequences, labels = [],[]
for action in actions:
    for sequence in range(no_sequences):
        window = [] #frames qe i kemi per ate sekuence 30 videos/sekuenca
        for frame_num in range(sequence_length):  
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res) #kryhet video 1 dhe i shtohet sekuencave ne rreshtin tjeter 
        sequences.append(window)
        labels.append(label_map[action])

In [26]:
X = np.array(sequences)

In [27]:
y = to_categorical(labels).astype(int)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.05)

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [ ]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir) #observimi i modelit derisa trajnohet

In [12]:
model = Sequential()
model.add(LSTM(64,return_sequences=True,activation='relu',input_shape=(30,1662))) #X.shape (90,30,1662) e marrim 30 dhe 1662 na duhet me shiku
model.add(LSTM(128,return_sequences=True,activation='relu'))
model.add(LSTM(64,return_sequences=False,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dense(32,activation='relu'))
model.add(Dense(actions.shape[0],activation='softmax'))

In [34]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['categorical_accuracy'])

In [ ]:
#mundemi me ndal edhe ma heret kur te shohim qe po performon mire modeli  
model.fit(X_train,y_train,epochs=140,callbacks=[tb_callback])

In [37]:
#parashikimet
res = model.predict(X_test)
actions[np.argmax(res[0])]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step


'C'

In [38]:
actions[np.argmax(y_test[0])]

'C'

In [3]:
#ruajtja e modelit/weights
model.save('action.h5')

NameError: name 'model' is not defined

In [26]:
#Nese e fshijme modelin shkojme e bejme rebuild tek ky cell model = Sequential() ...
#Pastaj e bejme compile dhe vazhdojm me load
model.load_weights('action.h5')

In [30]:
import tensorflow as tf
tf.keras.models.save_model(model, 'action.keras')

In [14]:
#Vleresimi se sa mire po performon modeli yne 
from sklearn.metrics import multilabel_confusion_matrix,accuracy_score

In [ ]:
yhat = model.predict(X_test)
ytrue = np.argmax(y_test,axis=1).tolist()
yhat = np.argmax(yhat,axis=1).tolist()

In [ ]:
multilabel_confusion_matrix(ytrue,yhat)
accuracy_score(ytrue,yhat)

In [27]:
colors = [(245,117,16),(117,245,16),(16,117,245)]
#shfaq te gjitha fjalet dhe prob. e tyre 
def prob_viz(res,actions,input_frame,colors):
    output_frame = input_frame.copy()
    for num,prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40),(int(prob*100),90+num*40),colors[num],-1)
        cv2.putText(output_frame,actions[num],(0,85+num*40),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)
    return output_frame

In [ ]:
#Testing in real time
#duke perdorur opencv marrim akses ne kamere
sequence = [] #marrja e frames qe pastaj me i bo pass dhe me parashiku
sentence = []
predictions = []
threshold = 0.4 #confidence matrix, i marrim rez vetem nese jane mbi nje vlere te caktuar
cap = cv2.VideoCapture(0)
# Set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        #read feed
        ret, frame = cap.read() #fotoja ne webcam
    
        # Make detection 
        image, results = mediapipe_detection(frame, holistic)
        print(results)

        # Draw landmarks
        draw_styled_landmarks(image,results)
        #Prediction logic 
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence,axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
    

            #Logjika vizualizimit
            if np.unique(predictions[-10:])[0] == np.argmax(res):            
                if res[np.argmax(res)] > threshold:
                    if len(sentence) > 0:
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])
    
            if len(sentence) > 5:
                sentence = sentence[-5:]
    
            image = prob_viz(res,actions,image,colors)

        cv2.rectangle(image, (0,0), (640,40), (245,117,16),-1)
        cv2.putText(image,' '.join(sentence),(3,30),
                   cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)
        
        #show to user/screen
        cv2.imshow("OpenCV Feed", image)
    
        #breaking gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break 
    cap.release()
    cv2.destroyAllWindows()
    # diskutabile duhet me provu ne rast se nuk mbyllet kamera mire - cv2.waitKey(1)